### Simiulation

Comparison of various MVP estimators including linear and nonlinear shrinkage methods. The mean and standard deviation (in parentheses) based on 1,000 replications of the Relative Risk ratio of estimated MVPs are reported. The number of assets is N = 84 and the sampling sizes are T = 50, 100
and 200. Monte Carlo with the assumption of normality.

In [ ]:
!pip install bs4 > NUL
!pip install yfinance > NUL
!pip install non-linear-shrinkage > NUL
!pip install tabulate > NUL

In [ ]:
import numpy as np
import pandas as pd
import requests
import bs4 as bs
import datetime
import random
import yfinance as yf
import pandas_datareader as web
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
from scipy.integrate import quad
import statsmodels.formula.api as sm
from scipy.stats import multivariate_normal
from scipy.linalg import inv
import nonlinshrink as nls
from sklearn.covariance import GraphicalLasso, GraphicalLassoCV, LedoitWolf, ShrunkCovariance, OAS

In [ ]:
sample_sizes = [50,100,200]
methods = ['PlugIn', 'linear_shrinkage', 'non_linear_shrinkage', 'equally_weighted']

def Risk_Ratio_RR(covRe_df, estimated_portfolio_weights):
    '''
    evaluation is based on risk ratio (R.R.) defined as the ratio between the standard
    deviation of the return of an estimated portfolio, w^, and the minimum standard
    deviation.
    '''
    given_covariance_matrix = covRe_df
    inverse_given_covariance_matrix = inv(covRe_df)
    identity_matrix = np.ones(len(covRe_df))

    # Nominator Risk R(W^)
    nominator_risk = estimated_portfolio_weights.T @ given_covariance_matrix @ estimated_portfolio_weights

    # Denominator Risk R(W*) -> Rmin
    denominator_risk = 1 /(identity_matrix @ inverse_given_covariance_matrix @ identity_matrix)

    return np.sqrt((nominator_risk / denominator_risk))


def prepare_multivariate_parameters(start_date=datetime.datetime(2004, 1, 1), end_date=datetime.datetime(2016, 1, 1),
                                     number_of_companies=None, seed=None, covariance_method='graphicallassocv',
                                     pre_downloaded_data=None):
    # Set seed for reproducibility
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # Getting the S&P 100 tickers from Wikipedia
    resp = requests.get('https://en.wikipedia.org/wiki/S%26P_100')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = [row.findAll('td')[0].text.replace('\n', '') for row in table.findAll('tr')[1:]]
    if seed is not None:
        np.random.seed(seed)
    if number_of_companies is not None:
        tickers = np.random.choice(tickers, number_of_companies, replace=False)

    # Downloading data from Yahoo finance
    if pre_downloaded_data is None:
        data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
        cleaned_data = data.dropna(axis='columns', how='any').dropna(axis='index', how='any')
    else:
        cleaned_data = pre_downloaded_data

    Re = (np.log(cleaned_data.iloc[:, :-1]) - np.log(cleaned_data.iloc[:, :-1].shift(1))).dropna()
    # Reading factor data from Fama-French
    df_factors = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start=start_date, end=end_date)[0]
    df_factors.rename(columns={'Mkt-RF': 'MKT'}, inplace=True)
    df_factors['MKT'] = df_factors['MKT'] / 100
    ##  Market return for S&P100 can also be retrived directly from Yahoo
    #market_returns = yf.download('^OEX', start=start_date, end=end_date)['Adj Close'].pct_change().dropna()

    # Compute CAPM and residuals
    coeffs = np.empty((Re.shape[1],1))
    residuals = np.empty((len(Re), 0))
    for i in range(Re.shape[1]):
        df_stk = Re.iloc[:, i]
        df_stock_factor = pd.merge(df_stk, df_factors, left_index=True, right_index=True)
        df_stock_factor['XsRet'] = df_stock_factor.iloc[:, 0] - df_stock_factor['RF']
        CAPM = sm.ols(formula='XsRet ~ MKT', data=df_stock_factor).fit(cov_type='HAC', cov_kwds={'maxlags': 1})
        coeffs[i] = CAPM.params[1]
        residuals = np.hstack((residuals, CAPM.resid.values.reshape(-1, 1)))

    # Estimation of covU, sigma2_m, and covRe

    # Compute covU based on the selected method
    if covariance_method == 'lw':
        lw = LedoitWolf().fit(residuals)
        covU = lw.covariance_
    elif covariance_method == 'sc':
        sc = ShrunkCovariance().fit(residuals)
        covU = sc.covariance_
    elif covariance_method == 'oas':
        oas = OAS().fit(residuals)
        covU = oas.covariance_
    elif covariance_method == 'pds':
        sc = ShrunkCovariance().fit(residuals)
        oas = OAS().fit(residuals)
        alpha = 0.5  # regularization parameter
        covU = alpha * np.abs(sc.covariance_) + (1 - alpha) * oas.covariance_
    elif covariance_method == 'graphicallassocv':
        model_cov = GraphicalLassoCV()
        covU = model_cov.fit(residuals).covariance_
    else:
        raise ValueError(f"Unknown covariance_method: {covariance_method}")
    sigma2_m = np.var(df_factors['MKT'])
    B = coeffs.reshape(-1, 1)
    covRe = sigma2_m * B @ B.T + covU  # Estimated covariance matrix for returns

    return covRe, len(covRe)

def generate_multivariate_returns2(covRe, sample_size):
    # Generate multivariate returns
    rv = multivariate_normal(mean=np.zeros(len(covRe)), cov=covRe)
    multivariate_returns = rv.rvs(size=sample_size)
    # Convert to DataFrame
    return_series = pd.DataFrame(multivariate_returns, columns=[f'Asset_{i}' for i in range(1, len(covRe) + 1)])

    return return_series


covRe, length_covRe = prepare_multivariate_parameters(covariance_method='pds')

# Create a DataFrame to hold the results
results_df = pd.DataFrame(columns=['Portfolio', 'T=50', 'T=100', 'T=200'])

# Loop through each method and sample size
for method in methods:
    row_data = [method]
    for sample_size in sample_sizes:
        risk_ratios = [] # List to store the risk ratios

        for _ in range(1000): # Run 1000 simulations
            return_series = generate_multivariate_returns2(covRe, sample_size=sample_size)
            identity_matrix = np.ones(len(covRe))
            sample_cov_matrix = return_series.cov()

            # Calculate the estimated portfolio weights based on the selected method
            if method == 'PlugIn':
                estimated_portfolio_weights = (inv(sample_cov_matrix) @ identity_matrix) / (identity_matrix.T @ inv(sample_cov_matrix) @ identity_matrix)
            elif method == 'linear_shrinkage':
                shrunk_cov = LedoitWolf().fit(return_series).covariance_
                estimated_portfolio_weights = (inv(shrunk_cov) @ identity_matrix) / (identity_matrix.T @ inv(shrunk_cov) @ identity_matrix)
            elif method == 'non_linear_shrinkage':
                sigma_tilde = nls.shrink_cov(return_series)
                estimated_portfolio_weights = (inv(sigma_tilde) @ identity_matrix) / (identity_matrix.T @ inv(sigma_tilde) @ identity_matrix)
            elif method == 'equally_weighted':
                cov_matrix = np.eye(len(covRe), len(covRe))
                estimated_portfolio_weights = (inv(cov_matrix) @ identity_matrix) / (identity_matrix.T @ inv(cov_matrix) @ identity_matrix)

            risk_ratio = Risk_Ratio_RR(covRe, estimated_portfolio_weights)
            risk_ratios.append(risk_ratio)

        average_risk_ratio = np.mean(risk_ratios)
        std_dev_risk_ratio = np.std(risk_ratios)

        row_data.append(f"{average_risk_ratio} ({std_dev_risk_ratio})")

    results_df.loc[len(results_df)] = row_data

table = tabulate(results_df, headers='keys', tablefmt='pipe', stralign='center')
print(table)

[*********************100%%**********************]  101 of 101 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['DOW']: Exception("%ticker%: Data doesn't exist for startDate = 1072933200, endDate = 1451624400")
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



|    |      Portfolio       |                    T=50                    |                   T=100                    |                   T=200                    |
|---:|:--------------------:|:------------------------------------------:|:------------------------------------------:|:------------------------------------------:|
|  0 |        PlugIn        |   50.97504963128382 (344.02438671557593)   |  2.4417100315376876 (0.39395618169259256)  | 1.3043380719892483 (0.055065274129960116)  |
|  1 |   linear_shrinkage   |  1.3166177336035694 (0.06674957834846579)  |  1.3320348248670821 (0.06111533112450627)  |  1.2088202774385117 (0.03319756207751076)  |
|  2 | non_linear_shrinkage | 1.1992791153546836 (0.041514089820499256)  |  1.1641108700994716 (0.03253357126267914)  |  1.1148936146566744 (0.01986082849963617)  |
|  3 |   equally_weighted   | 1.4938001886770529 (6.661338147750939e-16) | 1.4938001886770529 (6.661338147750939e-16) | 1.4938001886770529 (6.661338147750939e-16) |
